# Obtendo dados e indicadores fonanceiros da API FRED

Este projeto exemplifica a obtenção de dados da API do [_Federal Reserve od Saint Louis (FRED)_](https://fred.stlouisfed.org).

## Bibliotecas importadas

In [10]:
import datetime
from fredapi import Fred
import pandas_datareader as pdr

É possível acessar e realizar o _download_ dos dados da API FRED, por meio da biblioteca python [`fredapi`](https://github.com/mortada/fredapi). Além da instalação da referida bibliteca, deve-se ainda obter a chave pessoal de uso da API Fred:

In [4]:
fred = Fred(api_key_file='../keys/fredapi_key.txt')

Em pesquisa direta no endereço eletrônico do FRED por índices de preços ao consumidor (_Consumer Price Index_), encontramos o conjunto de dados _COnsumer Price Index: Total All Items for the United States_, cujo código para consulta é "CPALTT01USM657N" (ver figura abaixo).

![FRED Search](img/fred-search.png)

In [5]:
consumer_price_index = fred.get_series('CPALTT01USM657N')

In [7]:
consumer_price_index.tail()

2021-08-01    0.206591
2021-09-01    0.271597
2021-10-01    0.830812
2021-11-01    0.491343
2021-12-01    0.307252
dtype: float64

A fim de manipular os dados obtidos acima, é recomendado utilizar a biblioteca python [`pandas_datareader`](https://github.com/pydata/pandas-datareader), como mostrado abaixo:

In [11]:
# Definição das data de início e fim da requisição dos dados:
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2020, 9, 1)

In [13]:
# Download dos dados usando pandas_datareader
consumer_price_index = pdr.DataReader('CPALTT01USM657N', 'fred', start, end)

Note-se que o objeto retornado é um _dataframe_ passível de manipulação por meio da biblioteca Pandas:

In [14]:
consumer_price_index.tail()

,CPALTT01USM657N
DATE,
2020-05-01,0.001950
2020-06-01,0.547205
2020-07-01,0.505824
2020-08-01,0.315321
2020-09-01,0.139275


In [17]:
df_cpi = consumer_price_index.rename(columns={'CPALTT01USM657N': 'CPI'})
df_cpi.head()

,CPI
DATE,
2019-01-01,0.190660
2019-02-01,0.422705
2019-03-01,0.564136
2019-04-01,0.529500
2019-05-01,0.212876


## Produto Interno Bruto

O Produto Interno Bruto (PIB) pode ser calculado pelo lado da demanda por meio da seguinte fórmula:

$$PIB = C + I + G + (X – M)$$
em que:

- $C$: consumo das famílias em bens e serviços
- $I$: investimento das empresas
- $G$: gastos do governo em bens e serviços
- $X$: Exportações
- $I$: Importações

Obtenção de dados do PIB nort-americano entre 01-01-2019 e 01-09-2020:}

In [20]:
pib = pdr.DataReader('GDPC1', 'fred', start, end)
pib.head()

,GDPC1
DATE,
2019-01-01,18833.195
2019-04-01,18982.528
2019-07-01,19112.653
2019-10-01,19202.310
2020-01-01,18951.992


A fim de analizar a evolução do PIB norte-americano no período considerado, utilizamos o método `pct_change` dos _dataframes_ pandas para calcular a taxa de crescimento trimestral desse indicador:

In [21]:
pib['PIB %'] = pib.pct_change()
pib.head()

,GDPC1,PIB %
DATE,,
2019-01-01,18833.195,NaN
2019-04-01,18982.528,0.007929
2019-07-01,19112.653,0.006855
2019-10-01,19202.310,0.004691
2020-01-01,18951.992,-0.013036


## PIP Deflator

Fórmula: $$PIB_{deflator} = \dfrac{PIB_{nominal}}{PIB_{real}} \times 100$$

In [22]:
start = datetime.datetime(2017, 1, 1)
end = datetime.datetime(2017, 12, 31)

In [23]:
pib_nominal = pdr.DataReader(['GDP'], 'fred', start, end)
pib_nominal.head()

,GDP
DATE,
2017-01-01,19153.912
2017-04-01,19322.920
2017-07-01,19558.693
2017-10-01,19882.965


In [24]:
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2019, 12, 31)

In [25]:
pib_real = pdr.DataReader(['GDPC1'], 'fred', start, end)
pib_real.head()

,GDPC1
DATE,
2019-01-01,18833.195
2019-04-01,18982.528
2019-07-01,19112.653
2019-10-01,19202.310


In [26]:
pibn_sum = int(pib_nominal.sum())
pibr_sum = int(pib_real.sum())

In [27]:
pib_deflator = pibn_sum / pibr_sum * 100
pib_deflator

102.34861421253119

## Comparação do PIB de diferentes países

Nesta seção, vamos comparar o PIB do Brasil, China e Estados Unidos entre 01-01-2015 até 31-12-2015. O dados podem ser encontrados nos seguintes _links_:

* Brasil: https://fred.stlouisfed.org/series/BRAGDPNQDSMEI
* China: https://fred.stlouisfed.org/series/CHNGDPNQDSMEI
* Estados Unidos: https://fred.stlouisfed.org/series/USAGDPNQDSMEI

### Bibliotecas importadas

In [28]:
import pandas_datareader as pdr
import pandas as pd
import datetime
import matplotlib.pyplot as plt
plt.style.use('seaborn')

Matplotlib is building the font cache; this may take a moment.


In [29]:
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime(2020, 12, 31)

In [39]:
datasets = {'BRAGDPNQDSMEI': 'Brasil', 'CHNGDPNQDSMEI': 'China', 'USAGDPNQDSMEI': 'EUA'}

In [40]:
list(datasets.keys())

['BRAGDPNQDSMEI', 'CHNGDPNQDSMEI', 'USAGDPNQDSMEI']

In [43]:
pib_comparativo = pdr.DataReader(list(datasets.keys()), 'fred', start, end)
pib_comparativo.rename(columns=datasets, inplace=True)
pib_comparativo.head()

,Brasil,China,EUA
DATE,,,
2015-01-01,1.489495e+12,1.511379e+13,4.497837e+12
2015-04-01,1.485928e+12,1.685497e+13,4.548427e+12
2015-07-01,1.503836e+12,1.765977e+13,4.576740e+12
2015-10-01,1.516528e+12,1.925729e+13,4.583020e+12
2016-01-01,1.533247e+12,1.624100e+13,4.606326e+12


In [45]:
pib_change = pib_comparativo.pct_change() * 100
pib_change.dropna(inplace=True)
pib_change.head()

,Brasil,China,EUA
DATE,,,
2015-04-01,-0.239449,11.520472,1.124757
2015-07-01,1.205148,4.774853,0.622484
2015-10-01,0.843940,9.046097,0.137210
2016-01-01,1.102513,-15.663107,0.508546
2016-04-01,2.032381,11.697679,1.011169
